In [1]:
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

In [2]:
# Definir escopo de acesso ao Drive
scopes = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

In [3]:
# Criar as credenciais de acesso
creds = ServiceAccountCredentials.from_json_keyfile_name(
    filename="cursobigquery_OAuth_2-0.json",
    scopes=scopes
    )
# Criar o client
client = gspread.authorize(creds)

In [4]:
print(client)

#### Fixando o cabeçalho da Planilha

In [8]:
# Abrir a planilha
spreadsheet = client.open("Course_05-Movie_Data")

# Obter o ID da planilha
spreadsheet_id = spreadsheet.id

# URL da API
url = f"https://sheets.googleapis.com/v4/spreadsheets/{spreadsheet_id}:batchUpdate"

# ID da aba (gid) - normalmente é 0 para a primeira aba
sheet_id = spreadsheet.sheet1._properties['sheetId']

# Requisição para fixar o Header
payload = {
    "requests": [
        {
            "updateSheetProperties": {
                "properties":{
                    "sheetId": sheet_id,
                    "gridProperties": {"frozenRowCount": 1}
                },
                "fields": "gridProperties.frozenRowCount"
            }
        }
    ]
}

token = creds.get_access_token().access_token

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'spreadsheetId': '1ydeQcHioX7qh0oDJEZqb-zx7SpeCNZrbMNF738dLQ6Q', 'replies': [{}]}


#### Carregando a Planilha

In [9]:
planilha = client.open(
    title="Course_05-Movie_Data", 
    folder_id="1VeZ1jCJLUNXSDYpJrchCjPTtr5iZ7BKw")

In [10]:
planilha = planilha.get_worksheet(0)

In [11]:
planilha.get_all_records()

[{'Movie Title': '10 Cloverfield Lane',
  'Release Date': '2016-03-08',
  'Wikipedia URL': 'https://en.wikipedia.org/wiki/10_Cloverfield_Lane',
  'Genre': 'Thriller',
  'Director 1': 'Dan Trachtenberg',
  'Director 2': '',
  'Cast 1': 'Mary Elizabeth Winstead',
  'Cast 2': 'John Goodman',
  'Cast 3': 'John Gallagher',
  'Cast 4': '',
  'Cast 5': '',
  'Budget': '$15,000,000.00',
  'Revenue': '$108,300,000.00'},
 {'Movie Title': '13 Hours: The Secret Soldiers of Benghazi',
  'Release Date': '2016-01-15',
  'Wikipedia URL': 'https://en.wikipedia.org/wiki/13_Hours:_The_Secret_Soldiers_of_Benghazi',
  'Genre': 'Action',
  'Director 1': 'Michael Bay',
  'Director 2': '',
  'Cast 1': 'James Badge Dale',
  'Cast 2': 'John Krasinski',
  'Cast 3': 'Toby Stephens',
  'Cast 4': 'Pablo Schreiber',
  'Cast 5': 'Max Martini',
  'Budget': '$45,000,000.00',
  'Revenue': '$69,400,000.00'},
 {'Movie Title': '2 Guns',
  'Release Date': '2013-08-02',
  'Wikipedia URL': 'https://en.wikipedia.org/wiki/2_Gun

In [12]:
def exibir_planilha(planilha):
    dados = planilha.get_all_records()
    df = pd.DataFrame(dados)
    return df

In [13]:
# Ordenar a os dados pela coluna Release Date
planilha.sort((2, "asc"))

exibir_planilha(planilha)

,Movie Title,Release Date,Wikipedia URL,Genre,Director 1,Director 2,Cast 1,Cast 2,Cast 3,Cast 4,Cast 5,Budget,Revenue
0,The Devil Inside,2012-01-06,https://en.wikipedia.org/wiki/The_Devil_Inside...,Horror,William Brent Bell,,Suzan Crowley,Fernanda Andrade,Simon Quarterman,Evan Helmuth,,"$1,000,000.00","$101,800,000.00"
1,Red Tails,2012-01-20,https://en.wikipedia.org/wiki/Red_Tails,Drama,Anthony Hemingway,,Cuba Gooding Jr.,Nate Parker,David Oyelowo,Daniela Ruah,Terrence Howard,"$58,000,000.00","$50,400,000.00"
2,Declaration of War,2012-01-27,https://en.wikipedia.org/wiki/Declaration_of_W...,Romance,Valérie Donzelli,,Valérie Donzelli (director),,,,,"$1,500,000.00","$6,500,000.00"
3,Man on a Ledge,2012-01-27,https://en.wikipedia.org/wiki/Man_on_a_Ledge,Action,Asger Leth,,Sam Worthington,Ed Harris,Elizabeth Banks,Jamie Bell,Kyra Sedgwick,"$42,000,000.00","$46,200,000.00"
4,One for the Money,2012-01-27,https://en.wikipedia.org/wiki/One_for_the_Mone...,Crime,Julie Anne Robinson,,Katherine Heigl,Debbie Reynolds,Jason O'Mara,Daniel Sunjata,Sherri Shepherd,"$40,000,000.00","$36,900,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,Kubo and the Two Strings,2016-08-19,https://en.wikipedia.org/wiki/Kubo_and_the_Two...,Fantasy,Travis Knight,,Art Parkinson,Matthew McConaughey,Rooney Mara,Charlize Theron,Ralph Fiennes,"$60,000,000.00","$27,600,000.00"
504,War Dogs,2016-08-19,https://en.wikipedia.org/wiki/War_Dogs_(2016_f...,Crime,Todd Phillips,,Jonah Hill,Miles Teller,Ana de Armas,J. B. Blanc,,"$40,000,000.00","$42,700,000.00"
505,Don't Breathe,2016-08-26,https://en.wikipedia.org/wiki/Don%2527t_Breath...,Thriller,Fede Alvarez,,Dylan Minnette,Stephen Lang,Daniel Zovatto,,,"$9,900,000.00","$28,300,000.00"
506,Hands of Stone,2016-08-26,https://en.wikipedia.org/wiki/Hands_of_Stone,Biography,Jonathan Jakubowicz,,Édgar Ramírez,Robert De Niro,Usher,Ellen Barkin,Ana de Armas,"$20,000,000.00","$1,700,000.00"


In [14]:
# Ordenar a os dados pela coluna Release Date seguido da ordenação da coluna Movie Title
planilha.sort((2, 'asc'), (1, 'asc'))

exibir_planilha(planilha)

,Movie Title,Release Date,Wikipedia URL,Genre,Director 1,Director 2,Cast 1,Cast 2,Cast 3,Cast 4,Cast 5,Budget,Revenue
0,The Devil Inside,2012-01-06,https://en.wikipedia.org/wiki/The_Devil_Inside...,Horror,William Brent Bell,,Suzan Crowley,Fernanda Andrade,Simon Quarterman,Evan Helmuth,,"$1,000,000.00","$101,800,000.00"
1,Red Tails,2012-01-20,https://en.wikipedia.org/wiki/Red_Tails,Drama,Anthony Hemingway,,Cuba Gooding Jr.,Nate Parker,David Oyelowo,Daniela Ruah,Terrence Howard,"$58,000,000.00","$50,400,000.00"
2,Declaration of War,2012-01-27,https://en.wikipedia.org/wiki/Declaration_of_W...,Romance,Valérie Donzelli,,Valérie Donzelli (director),,,,,"$1,500,000.00","$6,500,000.00"
3,Man on a Ledge,2012-01-27,https://en.wikipedia.org/wiki/Man_on_a_Ledge,Action,Asger Leth,,Sam Worthington,Ed Harris,Elizabeth Banks,Jamie Bell,Kyra Sedgwick,"$42,000,000.00","$46,200,000.00"
4,One for the Money,2012-01-27,https://en.wikipedia.org/wiki/One_for_the_Mone...,Crime,Julie Anne Robinson,,Katherine Heigl,Debbie Reynolds,Jason O'Mara,Daniel Sunjata,Sherri Shepherd,"$40,000,000.00","$36,900,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,Kubo and the Two Strings,2016-08-19,https://en.wikipedia.org/wiki/Kubo_and_the_Two...,Fantasy,Travis Knight,,Art Parkinson,Matthew McConaughey,Rooney Mara,Charlize Theron,Ralph Fiennes,"$60,000,000.00","$27,600,000.00"
504,War Dogs,2016-08-19,https://en.wikipedia.org/wiki/War_Dogs_(2016_f...,Crime,Todd Phillips,,Jonah Hill,Miles Teller,Ana de Armas,J. B. Blanc,,"$40,000,000.00","$42,700,000.00"
505,Don't Breathe,2016-08-26,https://en.wikipedia.org/wiki/Don%2527t_Breath...,Thriller,Fede Alvarez,,Dylan Minnette,Stephen Lang,Daniel Zovatto,,,"$9,900,000.00","$28,300,000.00"
506,Hands of Stone,2016-08-26,https://en.wikipedia.org/wiki/Hands_of_Stone,Biography,Jonathan Jakubowicz,,Édgar Ramírez,Robert De Niro,Usher,Ellen Barkin,Ana de Armas,"$20,000,000.00","$1,700,000.00"


Utilizando a Função SORT

In [22]:
spreadsheet_PartyPlan = client.open(
    title="Course_05-Party_Plan_Spreadsheet", 
    folder_id="1VeZ1jCJLUNXSDYpJrchCjPTtr5iZ7BKw")

In [23]:
spreadsheet_PartyPlan = spreadsheet_PartyPlan.get_worksheet(0)

In [25]:
pd.DataFrame(spreadsheet_PartyPlan.get_all_records())

,Guest Name,Table Number,Dietary Restriction,Sent Invitation,Row Index
0,Jack,2,Vegetarian,No,5
1,Omar,1,None,Yes,4
2,Aida,3,None,Yes,3
3,Nancy,3,None,No,2
4,Jianyu,2,Vegetarian,Yes,1
5,,,,,
6,,,,,
7,Guest Name,Table Number,Dietary Restriction,Sent Invitation,Formula
8,Omar,1,None,Yes,"=SORT(A2:D6, 2, TRUE)"
9,Jack,2,Vegetarian,No,


In [46]:
spreadsheet_PartyPlan.update([['Guest Name Sort', 'Table Number Sort', 
                               'Dietary Restriction Sort', 'Sent Invitation Sort', 
                               'Row Index Sort']], "G1:L1")

{'spreadsheetId': '1oP2L6DTJpAfSgnDyi8h-97dO6bf9BgpEtWolJderJFk',
 'updatedRange': "'Party Plan'!G1:K1",
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

In [43]:
spreadsheet_PartyPlan.update([['=SORT(A2:E6; 2; TRUE)']], "G2", raw=False)

{'spreadsheetId': '1oP2L6DTJpAfSgnDyi8h-97dO6bf9BgpEtWolJderJFk',
 'updatedRange': "'Party Plan'!G2",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [47]:
pd.DataFrame(spreadsheet_PartyPlan.get_all_records())

,Guest Name,Table Number,Dietary Restriction,Sent Invitation,Row Index,,Guest Name Sort,Table Number Sort,Dietary Restriction Sort,Sent Invitation Sort,Row Index Sort
0,Jack,2,Vegetarian,No,5,,Omar,1,None,Yes,4
1,Omar,1,None,Yes,4,,Jack,2,Vegetarian,No,5
2,Aida,3,None,Yes,3,,Jianyu,2,Vegetarian,Yes,1
3,Nancy,3,None,No,2,,Aida,3,None,Yes,3
4,Jianyu,2,Vegetarian,Yes,1,,Nancy,3,None,No,2
5,,,,,,,,,,,
6,,,,,,,,,,,
7,Guest Name,Table Number,Dietary Restriction,Sent Invitation,Formula,,,,,,
8,Omar,1,None,Yes,"=SORT(A2:D6, 2, TRUE)",,,,,,
9,Jack,2,Vegetarian,No,,,,,,,
